In [1]:
import requests
import lxml.html
from selenium.webdriver import Chrome
import pandas as pd

In [2]:
browser = Chrome()

In [3]:
url = 'https://kr.hotels.com/search.do?resolved-location=COUNTRY%3A10233090%3AUNKNOWN%3AUNKNOWN&f-star-rating=5,4,3&destination-id=10233090&q-destination=%ED%95%9C%EA%B5%AD&q-check-in=2020-05-13&q-check-out=2020-05-14&q-rooms=1&q-room-0-adults=2&q-room-0-children=0&sort-order=STAR_RATING_HIGHEST_FIRST'

In [4]:
requests.get(url)

<Response [200]>

In [5]:
browser.get(url)

In [6]:
import time
SCROLL_PAUSE_TIME = 5

# Get scroll height
last_height = browser.execute_script("return document.body.scrollHeight-1000")         

while True:
    # Scroll down to bottom                                                      
    browser.execute_script("window.scrollTo(0, document.body.scrollHeight-1000);")

    # Wait to load page
    time.sleep(SCROLL_PAUSE_TIME)                                                
    browser.execute_script("window.scrollTo(0, document.body.scrollHeight-1000);")  
    time.sleep(SCROLL_PAUSE_TIME)

    # Calculate new scroll height and compare with last scroll height            
    new_height = browser.execute_script("return document.body.scrollHeight")

    if new_height == last_height:                                               
        break

    last_height = new_height


In [7]:
hotel_reviews = []

In [8]:
hotels = browser.find_elements_by_css_selector('a.property-name-link')

In [75]:
#호텔 사이트
for i in range(len(hotels)) :
    time.sleep(1)
    hotels[i].click()
    time.sleep(2)
    browser.switch_to_window(browser.window_handles[1])   # 새로운 탭으로 이동 
    time.sleep(1)
    hotel_name = browser.find_element_by_tag_name('h1').text
    time.sleep(2)
    try :
        browser.find_elements_by_css_selector('a.total-reviews')[0].click() # 리뷰 이동
        time.sleep(1)
    except : 
        browser.close()
        browser.switch_to_window(browser.window_handles[0])
        continue
        
        
    while 1:
        try :
            reviews = browser.find_elements_by_css_selector('div.brand-reviews-listing>div.review-card>blockquote')
            ratings = browser.find_elements_by_css_selector('div.brand-reviews-listing>div.review-card> div.card-header>span.rating >span.rating-score')
            for j in range(len(reviews)) :
                review = reviews[j].text
                rating = ratings[j].text
                hotel_reviews.append([hotel_name,review,rating])
            browser.find_element_by_css_selector('div.pagination > div > a.cta.cta-secondary.cta-next').click()         # 다음페이지 
            time.sleep(3)
        except :
            browser.close()
            break
    browser.switch_to_window(browser.window_handles[0])



C:\Users\dhgus\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: use driver.switch_to.window instead
  
C:\Users\dhgus\Anaconda3\lib\site-packages\ipykernel_launcher.py:32: DeprecationWarning: use driver.switch_to.window instead
C:\Users\dhgus\Anaconda3\lib\site-packages\ipykernel_launcher.py:15: DeprecationWarning: use driver.switch_to.window instead
  from ipykernel import kernelapp as app


In [76]:
result = pd.DataFrame(hotel_reviews, columns = ['호텔이름','리뷰','평점'])

In [77]:
result.to_excel('.xlsx', encoding = 'utf-8',index = False)